In [1]:
from GASU_env import GASU
from GASU_optimizer import optimize_GASU

In [ ]:
# Episode Length: T
T = 31
state_horizon = 30
action_horizon = 1
simulation_days = T + state_horizon - action_horizon       # Total simulation days = 60
"""
at the start of each day (1-31): 
    - 30 days of demand data
    - 30 days of price data

at the start of 31st day: we have data of 31st day itself and 29 coming days (total 30 days, making simulation days = 60)

i.e., episode length (T) = 31 days (1-31)
"""
env = GASU(T, state_horizon, action_horizon)
# Rendering essential information from environment
compressors = env.compressor_info()       # Provides the infromation compressors and its initial state in dict format
external_purchase_price = env.get_external_purchase_price()    # Provides the dict of external purchase price

# initializing the optimizer by feeding the essential information from environment
optimizer = optimize_GASU(state_horizon, action_horizon, compressors)
optimizer._update_external_purchase_price(external_purchase_price)         # __Update the external purchase price in optimizer__

### Simulation for 31 days (with optimal action)

In [3]:
env.reset()
opt_episode_reward = 0
simulation_episode_reward = 0
optimal_episode_cost = 0
state = env.render()
done = False
while done is False:
    optimizer.update_state(state)
    objective_value = optimizer.solve()
    opt_cost = optimizer.action_horizon_cost()
    opt_action = optimizer._fetch_optimal_action_for_action_horizon()
    # feed action to environment
    state, reward, done, truncated, info = env.step(opt_action)
    opt_episode_reward += opt_cost
    
print(f"Optimal Episode Reward: {opt_episode_reward}")
print(f"Final Episode Reward: {reward}")

Optimal Episode Reward: 955866.6748966866
Final Episode Reward: -955883.0184969649
